# Regrid CloudSat data to horizontal grid


# Table of Contents
<ul>
<li><a href="#introduction">1. Introduction</a></li>
<li><a href="#data_wrangling">2. Data Wrangling</a></li>
<li><a href="#exploratory">3. Exploratory Data Analysis</a></li>
<li><a href="#conclusion">4. Conclusion</a></li>
<li><a href="#references">5. References</a></li>
</ul>

# 1. Introduction <a id='introduction'></a>

**Questions**


> **_NOTE:_** The pressure grid was created in the  [Jupyter Notebook](./get_pressure_grid.ipynb) to calculate the daily means of CloudSat variables.

# 2. Data Wrangling <a id='data_wrangling'></a>



- Time period: 2007 - 2010
- time resolution: daily atmospheric overpasses 
- Variables:
  
|              Long name                   |      Units    |  Dimension |
| :---------------------------------------:| -------------:|--------:|






In [5]:
import os
import pathlib
import sys
import socket
hostname = socket.gethostname()

abs_path = str(pathlib.Path(hostname).parent.absolute())
WORKDIR = abs_path[:- (len(abs_path.split('/')[-2] + abs_path.split('/')[-1])+1)]


if "mimi" in hostname:
    print(hostname)
    DATA_DIR = "/scratch/franzihe/"
    # FIG_DIR = "/uio/kant/geo-metos-u1/franzihe/Documents/Figures/ERA5/"
elif "glefsekaldt" in hostname: 
    DATA_DIR = "/home/franzihe/Data/"
    # FIG_DIR = "/home/franzihe/Documents/Figures/ERA5/"

INPUT_DATA_DIR = os.path.join(DATA_DIR, 'input')
OUTPUT_DATA_DIR = os.path.join(DATA_DIR, 'output')
UTILS_DIR = os.path.join(WORKDIR, 'utils')

sys.path.append(UTILS_DIR)
# make figure directory
# try:
#     os.mkdir(FIG_DIR)
# except OSError:
#     pass

mimi.uio.no


## Import python packages
- `Python` environment requirements: file [globalsnow.yml](../globalsnow.yml) 
- load `python` packages from [imports.py](../utils/imports.py)
- load `functions` from [functions.py](../utils/functions.py)

In [6]:
# supress warnings
import warnings
warnings.filterwarnings('ignore') # don't output warnings

# import packages
from imports import(xr, ccrs, cy, plt, glob, fct, np, datetime, timedelta, h5py, da)


In [7]:
# reload imports
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Open CloudSat variables
Get the data requried for the analysis. Beforehand we downloaded the [ECMWF-AUX files](https://www.cloudsat.cira.colostate.edu/data-products/ecmwf-aux) with the script provided on the [CloudSat webpage](https://cswww.cira.colostate.edu/code_library/cloudsat_ftp.plx).



The CloudSat data is located in the folder `/input/CloudSat/` and individual folders for the CloudSat product. The 2C-SNOW-PROFILEs are in `/2C-SNOW-PROFILE.P1_R05/` while the ECMWF-AUX files are in `/ECMWF-AUX.P_R05/`. Each year has extra folders for the day of the year, where all CloudSat granules can be found of that specific day. 


In [9]:
cs_in = os.path.join(INPUT_DATA_DIR, 'CloudSat/2C-SNOW-PROFILE.P1_R05')
cs_out = os.path.join(OUTPUT_DATA_DIR, 'CloudSat/2C-SNOW-PROFILE.P1_R05')
# make output data directory
try:
    os.mkdir(cs_out)
except OSError:
    pass

In [3]:
ff_cs = sorted(glob(cs_in+'/2007/*/*.h5'))

In [4]:
GEOLOC_VARNAMES = ["DEM_elevation","Height", "Latitude", "Longitude"]
DATA_VARNAMES = ["snow_water_content", "snowfall_rate", "snowfall_rate_sfc"]
DATA_VARNAMES_EC = ["Pressure", "Temperature"]
_VARNAMES = np.append(DATA_VARNAMES, DATA_VARNAMES_EC)


In [5]:
# ds = xr.Dataset()
# ds['Pressure_grid'] = xr.open_dataset('{}ECMWF-AUX.P_R05/pressure_grid.nc'.format(cs_out))['Pressure_grid']

In [6]:
# # load era grid
# ds_era = xr.open_dataset('{}/clwc_Amon_ERA5_198501_198912.nc'.format(era_in, ))
# # ds_era = ds_era.isel(time = month-1).squeeze().drop('clwc')
# ds_era = ds_era.drop('clwc')

In [16]:
datasets = []

for file in ff_cs[0:1]:
    year = int(file.split('/')[-1].split('_')[0][0:4])
    doy = int(file.split('/')[-1].split('_')[0][4:7])  # day of the year
    tt = datetime(year, 1, 1) + timedelta(doy - 1)
    
    
    file_ec = sorted(glob('{}ECMWF-AUX.P_R05/{}/{}/{}_*.h5'.format(cs_in, year, doy, file.split('/')[-1].split('_')[0])))[0]
    for month in range(6, 7):
        if tt.month == month:
            # ds = xr.Dataset()
            ds = xr.Dataset()
            ds['Pressure_grid'] = xr.open_dataset('{}ECMWF-AUX.P_R05/pressure_grid.nc'.format(cs_out))['Pressure_grid']

            h5file = h5py.File(file, "r")   # open h5 file
            ds['Profile_time'] = fct.get_profile_times(h5file['2C-SNOW-PROFILE'])

            for var in GEOLOC_VARNAMES:
                ds[var] = fct.get_geoloc_var(h5file['2C-SNOW-PROFILE'], var )

            for var in DATA_VARNAMES:
                ds[var] = fct.get_data_var(h5file['2C-SNOW-PROFILE'], var)

            # load ECMWF-AUX file
            h5file_ec = h5py.File(file_ec, "r")
            for var in DATA_VARNAMES_EC:
                ds[var] = fct.get_data_var(h5file_ec['ECMWF-AUX'], var)

                if var == 'Pressure':
                    ds[var] = ds[var]/100
                    ds[var].attrs = {'units': ds_era['level'].attrs['units'], 'longname':ds_era['level'].attrs['long_name']}

            for var in _VARNAMES:
                if (len(ds[var].dims)) == 2:
                    ds[var +'_pregrid'] = xr.DataArray(data=da.full(shape = (ds[var].shape), fill_value=np.nan, chunks=(len(ds['nray'])/2, len(ds['nbin']))), dims= list(ds[var].dims), coords=[ds[var]['nray'].values, ds[var]['nbin']])
                    # Before performing the average within a grid cell we will bring the Profiles on a common pressure grid which we created in get_pressure_grid.py
                    for nray in range(len(ds.nray)):
                        for nbin in range(len(ds.nbin)):
                            dmin= int(abs(ds['Pressure'].isel(nray = nray) - ds['Pressure_grid'].isel(nbin = nbin)).idxmin(dim = 'nbin'))
                            ds[var +'_pregrid'][nray, nbin] =  ds[var].isel(nray = nray, nbin = dmin)


            datasets.append(ds)

            

    ds_cs = xr.concat(datasets,dim='nray')

# # add common pressure grid to dataset
# ds_cs['Pressure_grid'] = xr.open_dataset('{}ECMWF-AUX.P_R05/pressure_grid.nc'.format(cs_out))['Pressure_grid']




KeyboardInterrupt: 

In [ ]:
# # convert ECMWF pressure to hPa
# ds_cs['Pressure'] = ds_cs['Pressure']/100
# ds_cs['Pressure'].attrs = {'units': ds_era['level'].attrs['units'], 'longname':ds_era['level'].attrs['long_name']}


In [ ]:
# # create dataset with same isobaric pressure levels
# ds_pres = ds_cs.drop(('DEM_elevation', 'Height', ))
# # create array

# for var in DATA_VARNAMES:
#     if (len(ds_cs[var].dims)) == 2:
#         ds_pres[var] = xr.DataArray(data=da.full(shape = (ds_cs[var].shape), fill_value=np.nan, chunks=(len(ds_cs['nray'])/2, len(ds_cs['nbin']))), dims= list(ds_cs[var].dims), coords=list(ds_cs[var].indexes.values()))


#         # Before performing the average within a grid cell we will bring the Profiles on a common pressure grid which we created in get_pressure_grid.py
#         for nray in range(len(ds_cs.nray)):
#             for nbin in range(len(ds_cs.nbin)):
                
#                 dmin= int(abs(ds_cs['Pressure'].isel(nray = nray) - ds_cs['Pressure_grid'].isel(nbin = nbin)).idxmin(dim = 'nbin'))
#                 ds_pres[var][nray, nbin] =  ds_cs[var].isel(nray = nray, nbin = dmin)

In [ ]:
lat_lon = xr.Dataset()

# create array 
for var in DATA_VARNAMES:
    if len(ds_pres[var].dims) == 1:
        lat_lon[var] = xr.DataArray(data=da.full(shape = (ds_era['longitude'].shape[0], ds_era['latitude'].shape[0], ), fill_value=np.nan, chunks=(ds_era['longitude'].shape[0]/2, ds_era['latitude'].shape[0])), 
                                                               dims=[ds_era['longitude'].dims[0], ds_era['latitude'].dims[0]], 
                                                               coords=[ds_era['longitude'].values, ds_era['latitude'].values])
    if len(ds_pres[var].dims) == 2:
        lat_lon[var] = xr.DataArray(data=da.full(shape = (ds_era['longitude'].shape[0], ds_era['latitude'].shape[0], ds_pres['Pressure_grid'].shape[0]), fill_value=np.nan, chunks=(ds_era['longitude'].shape[0]/2, ds_era['latitude'].shape[0], ds_pres['Pressure_grid'].shape[0])), 
                                                               dims=[ds_era['longitude'].dims[0], ds_era['latitude'].dims[0], ds_pres['Pressure_grid'].dims[0]], 
                                                               coords=[ds_era['longitude'].values, ds_era['latitude'].values, ds_pres['Pressure_grid'].values])

    # Calculate the average value within a grid box
    for i in range(len(lat_lon.longitude)+1):
        # loop through longitude
        if i == len(lat_lon.longitude):
            lon1 = 360
        else:
            lon1 = lat_lon['longitude'].isel(longitude=slice(i, i+2)).mean().values.tolist()
        if i == 0:
            lon0 = lat_lon['longitude'].isel(longitude=slice(i, i+1)).mean().values.tolist()
        else:
            lon0 = lat_lon['longitude'].isel(longitude=slice(i-1, i+1)).mean().values.tolist()

        # loop through latitude
        for k in range( len(lat_lon.latitude)):
            lat0 = lat_lon['latitude'].isel(latitude = slice(k, k+2)).mean().values.tolist()
            if k == 0:
                lat1 = lat_lon['latitude'].isel(latitude = slice(k, k+1)).mean().values.tolist()
            else:
                lat1 = lat_lon['latitude'].isel(latitude = slice(k-1, k+1)).mean().values.tolist()
            
            bbox = (lon0, lon1, lat0, lat1)
            # print(bbox)

            filter = ((ds_pres['Longitude'] > bbox[0]) & (ds_pres['Longitude'] < bbox[1]) & \
                (ds_pres['Latitude'] > bbox[2]) & (ds_pres['Latitude'] < bbox[3]))

            if len(ds_pres[var].dims) == 1:
                lat_lon[var][i, k] = ds_pres[var].where(filter).mean('nray',skipna=True,keep_attrs=True)
            if len(ds_pres[var].dims) == 2:
                lat_lon[var][i, k,:] = ds_pres[var].where(filter).mean('nray',skipna=True,keep_attrs=True)

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111, projection=ccrs.Robinson())
ax.coastlines()
# ax.set_global()
# ax.set_extent([-30, 50, 50, 85], crs=ccrs.PlateCarree())
ax.set_extent([-10, 30, 25, 85], crs=ccrs.PlateCarree())

ax.plot(combined['Longitude'].where(filter), combined['Latitude'].where(filter), linewidth=2, transform=ccrs.PlateCarree());